In [2]:
import osmnx as ox
import networkx as nx
from geopy.geocoders import Nominatim
import folium
from itertools import permutations
import pandas as pd
from typing import List, Tuple, Dict
import time


class DeliveryRouter:
    """
    A class to optimize delivery routes for hamster supply deliveries in Dallas
    using OpenStreetMap data.
    """
    
    def __init__(self):
        """Initialize the router with Dallas map data"""
        self.geolocator = Nominatim(user_agent="hamster_delivery_router")
        print("Loading Dallas street network...")
        self.dallas_graph = ox.graph_from_place('Dallas, Texas, USA', network_type='drive')
        self.dallas_graph_proj = ox.project_graph(self.dallas_graph)
        
    def validate_address(self, address: str) -> Tuple[bool, tuple]:
        """Validate a Dallas address and return its coordinates"""
        try:
            location = self.geolocator.geocode(address, timeout=10)
            if location and "Dallas" in location.address:
                return True, (location.latitude, location.longitude)
            return False, None
        except Exception as e:
            print(f"Error validating address {address}: {str(e)}")
            return False, None
            
    def calculate_route(self, start_coords: Tuple[float, float], 
                       end_coords: Tuple[float, float]) -> Tuple[float, List[Tuple[float, float]]]:
        """Calculate the shortest route between two points and return distance and path"""
        start_node = ox.nearest_nodes(self.dallas_graph, start_coords[1], start_coords[0])
        end_node = ox.nearest_nodes(self.dallas_graph, end_coords[1], end_coords[0])
        
        try:
            route = nx.shortest_path(self.dallas_graph_proj, start_node, end_node, weight='length')
            distance = sum(ox.utils_graph.get_route_edge_attributes(self.dallas_graph_proj, route, 'length'))
            
            # Get the actual path coordinates for mapping
            path_coords = []
            for node in route:
                y = self.dallas_graph.nodes[node]['y']
                x = self.dallas_graph.nodes[node]['x']
                path_coords.append((y, x))
                
            return distance, path_coords
        except nx.NetworkXNoPath:
            return float('inf'), []

    def optimize_route_fast(self, addresses: List[str]) -> Tuple[List[str], List[float], List[List[Tuple[float, float]]]]:
        """Find an optimized delivery route using nearest neighbor with 2-opt improvement"""
        # Validate addresses and get coordinates
        locations = []
        for addr in addresses:
            valid, coords = self.validate_address(addr)
            if valid:
                locations.append((addr, coords))
            else:
                raise ValueError(f"Invalid address: {addr}")

        # Warehouse is always first and last
        warehouse = locations[0]
        delivery_points = locations[1:]
        
        # Nearest neighbor construction
        route = [warehouse]
        remaining_points = delivery_points.copy()
        current_point = warehouse
        
        while remaining_points:
            # Find nearest unvisited point
            nearest = min(remaining_points, 
                        key=lambda x: self.calculate_route(current_point[1], x[1])[0])
            route.append(nearest)
            current_point = nearest
            remaining_points.remove(nearest)
        
        route.append(warehouse)  # Return to warehouse
        
        # 2-opt improvement
        improved = True
        while improved:
            improved = False
            for i in range(1, len(route) - 2):
                for j in range(i + 1, len(route) - 1):
                    # Calculate current segment distances
                    d1, _ = self.calculate_route(route[i-1][1], route[i][1])
                    d2, _ = self.calculate_route(route[j][1], route[j+1][1])
                    current_distance = d1 + d2

                    # Calculate new segment distances if we reverse the path
                    d3, _ = self.calculate_route(route[i-1][1], route[j][1])
                    d4, _ = self.calculate_route(route[i][1], route[j+1][1])
                    new_distance = d3 + d4

                    if new_distance < current_distance:
                        # Reverse the segment between i and j
                        route[i:j+1] = reversed(route[i:j+1])
                        improved = True
                        break
                if improved:
                    break
        
        # Calculate final distances and paths
        total_distance = 0
        paths = []
        
        for i in range(len(route) - 1):
            distance, path = self.calculate_route(route[i][1], route[i+1][1])
            total_distance += distance
            paths.append(path)
        
        return [loc[0] for loc in route], total_distance, paths

    def optimize_route_simple(self, addresses: List[str]) -> Tuple[List[str], float]:
        """
        Simple and fast route optimization using nearest neighbor approach
        Returns optimized route and total distance
        """
        print("Starting route optimization...")
        
        # Convert addresses to coordinates and validate
        locations = {}
        for addr in addresses:
            valid, coords = self.validate_address(addr)
            if valid:
                locations[addr] = coords
            else:
                print(f"Skipping invalid address: {addr}")
                continue
        
        # Start from warehouse (first address)
        current_addr = addresses[0]
        unvisited = set(addresses[1:])  # All addresses except warehouse
        optimized_route = [current_addr]
        total_distance = 0
        
        # Find nearest neighbor until all addresses are visited
        while unvisited:
            print(f"Finding next stop... {len(unvisited)} remaining")
            # Find closest unvisited address
            next_addr = min(
                unvisited,
                key=lambda x: self.calculate_route(
                    locations[current_addr],
                    locations[x]
                )[0]
            )
            
            # Calculate and add distance
            distance, _ = self.calculate_route(
                locations[current_addr],
                locations[next_addr]
            )
            total_distance += distance
            
            # Add to route and update current position
            optimized_route.append(next_addr)
            unvisited.remove(next_addr)
            current_addr = next_addr
        
        # Return to warehouse
        distance, _ = self.calculate_route(
            locations[current_addr],
            locations[addresses[0]]
        )
        total_distance += distance
        optimized_route.append(addresses[0])
        
        # Print human-readable summary
        print("\nRoute Summary:")
        print(f"Total Stops: {len(optimized_route)-1}")  # -1 because warehouse is counted twice
        print(f"Total Distance: {total_distance/1000:.2f} km ({total_distance*0.000621371:.2f} miles)")
        print("\nOptimized Route:")
        for i, addr in enumerate(optimized_route[:-1]):  # Skip last warehouse entry
            print(f"Stop {i}: {addr}")
        
        return optimized_route, total_distance
   
    def create_visualization(self, addresses: List[str], total_distance: float, 
                           paths: List[List[Tuple[float, float]]]) -> None:
        """Create and save route visualization"""
        # Create base map centered on Dallas
        dallas_center = [32.7767, -96.7970]
        m = folium.Map(location=dallas_center, zoom_start=12)
        
        # Add markers
        # a loop iterates through the addresses list, and each address is validated using self.validate_address, which checks the address formats and brings back coordinates
        # markers are added to the map using folium.marker, the first marker is colored red to distinguish it as the starting point, and subsequent markers are colored blue to denote regular stops
        # a popup on each marker displays the stop number and address
        # this section visually represents all stops on the map, making it easy to identify locations and their sequence
        for i, addr in enumerate(addresses[:-1]):  # Skip last duplicate warehouse
            coords = self.validate_address(addr)[1]
            color = 'red' if i == 0 else 'blue'
            folium.Marker(
                coords,
                popup=f"Stop {i}: {addr}",
                icon=folium.Icon(color=color)
            ).add_to(m)
        
        # Add route paths
        # For each path,  blue polyline is drawn on the map using folium.Polyline, wweight sets  the line thickness and opacity makes the line transparent for better readability
        # this section shows the optimized path
        for path in paths:
            folium.PolyLine(
                path,
                weight=2,
                color='blue',
                opacity=0.8
            ).add_to(m)
        
        # Save map
        # the folium mp object m is saved as an HTML file named optimized_route.html
        m.save('optimized_route.html')
        
        # Print route details
        # A loop iterates through consecutive pairs of addresses in the optimized list
        # For each pai the shortest path is calculated using NetworkX, and geographic data from OSMnx
        # The total segment distance is calculated by summing the length of the edges in the path, and distances are converted from miles to kms and dispayed
        print("\nOptimized Route with Distances:")
        for i in range(len(addresses) - 1):
            distance_km = sum(ox.utils_graph.get_route_edge_attributes(
                self.dallas_graph_proj, 
                nx.shortest_path(self.dallas_graph_proj,
                    ox.nearest_nodes(self.dallas_graph,
                        self.validate_address(addresses[i])[1][1],
                        self.validate_address(addresses[i])[1][0]),
                    ox.nearest_nodes(self.dallas_graph,
                        self.validate_address(addresses[i+1])[1][1],
                        self.validate_address(addresses[i+1])[1][0])),
                'length')) / 1000
            distance_mi = distance_km * 0.621371
            print(f"{addresses[i]} -> {addresses[i+1]}: {distance_km:.2f} km ({distance_mi:.2f} mi)")
            
        # The total distance for the optimized route is printed in both ks and miles
        print(f"Total distance: {total_distance/1000:.2f} km ({total_distance*0.000621371:.2f} mi)")
        
    def display_route_info(self, addresses: List[str], total_distance: float):
        """Display route information directly in the notebook"""
        print("\n=== OPTIMIZED DELIVERY ROUTE ===")
        print("\nRoute Sequence:")
        for i, addr in enumerate(addresses[:-1]):  # Skip last duplicate warehouse
            print(f"Stop {i:2d}: {addr}")
        
        print("\nSegment Distances:")
        for i in range(len(addresses) - 1):
            start_addr = addresses[i]
            end_addr = addresses[i + 1]
            
            # Calculate segment distance
            start_coords = self.validate_address(start_addr)[1]
            end_coords = self.validate_address(end_addr)[1]
            distance, _ = self.calculate_route(start_coords, end_coords)
            
            # Convert to km and miles
            distance_km = distance / 1000
            distance_mi = distance_km * 0.621371
            
            print(f"Segment {i:2d}: {distance_km:6.2f} km ({distance_mi:6.2f} mi)")
            print(f"         {start_addr} -> {end_addr}")

        # Print total distance
        total_km = total_distance / 1000
        total_mi = total_km * 0.621371
        print("\nTotal Route Distance:")
        print(f"  {total_km:.2f} kilometers")
        print(f"  {total_mi:.2f} miles")
     
    def optimize_route_faster(self, addresses: List[str]) -> Tuple[List[str], float]:
        """
        Faster route optimization using pre-calculated distances
        """
        print("Starting route optimization...")
        
        # Convert addresses to coordinates and validate (do this once)
        locations = {}
        for addr in addresses:
            valid, coords = self.validate_address(addr)
            if valid:
                locations[addr] = coords
            else:
                print(f"Skipping invalid address: {addr}")
                continue
        
        # Pre-calculate all distances between points
        distances = {}
        print("Pre-calculating distances...")
        for addr1 in addresses:
            distances[addr1] = {}
            for addr2 in addresses:
                if addr1 != addr2:
                    dist, _ = self.calculate_route(locations[addr1], locations[addr2])
                    distances[addr1][addr2] = dist
        
        # Start from warehouse
        current_addr = addresses[0]
        unvisited = set(addresses[1:])
        optimized_route = [current_addr]
        total_distance = 0
        
        # Find nearest neighbor using pre-calculated distances
        while unvisited:
            # Find closest unvisited address using cached distances
            next_addr = min(
                unvisited,
                key=lambda x: distances[current_addr][x]
            )
            
            # Add distance from cache
            total_distance += distances[current_addr][next_addr]
            
            # Update route
            optimized_route.append(next_addr)
            unvisited.remove(next_addr)
            current_addr = next_addr
        
        # Return to warehouse using cached distance
        total_distance += distances[current_addr][addresses[0]]
        optimized_route.append(addresses[0])
        
        # Print summary
        print("\nRoute Summary:")
        print(f"Total Stops: {len(optimized_route)-1}")
        print(f"Total Distance: {total_distance/1000:.2f} km ({total_distance*0.000621371:.2f} miles)")
        print("\nOptimized Route:")
        for i, addr in enumerate(optimized_route[:-1]):
            print(f"Stop {i}: {addr}")
    
        return optimized_route, total_distance


In [3]:
# Initialize router
router = DeliveryRouter()

# Define addresses (warehouse first)

addresses = [ "5351 Fults Blvd., Dallas, TX", "4078 Creekdale Drive, Dallas, TX", "7526 Morton Street, Dallas, TX", "2418 Montalba Avenue, Dallas, TX", "1412 Moran Drive, Dallas, TX", "6211 Annapolis Lane, Dallas, TX", "8814 Redondo Drive, Dallas, TX", "2925 Lenway Street, Dallas, TX", "3203 Bertrand Avenue, Dallas, TX", "7042 Desco Drive, Dallas, TX", "8332 Moorcroft Drive, Dallas, TX", "3724 Purdue Street, Dallas, TX", "7616 Dandy Lane, Dallas, TX", "1249 Olden Street, Dallas, TX", "1014 Delaware Avenue, Dallas, TX", "7318 Crownrich Lane, Dallas, TX", "1634 Engle Avenue, Dallas, TX", "6218 Velasco Avenue, Dallas, TX", "1500 Marilla Street, Dallas, TX", "534 Elkhart Avenue, Dallas, TX", "11322 Cactus Lane, Dallas, TX", "8855 Liptonshire, Dallas, TX", "10223 Clary Drive, Dallas, TX", "9731 Champa Drive, Dallas, TX", "11635 Sahara Way, Dallas, TX", "12167 Ridgelake Drive, Dallas, TX", "11008 Ridgemeadow Drive, Dallas, TX", "10357 Bel Aire, Dallas, TX" ]



# Get optimized route
route, distance = router.optimize_route_faster(addresses)

Loading Dallas street network...
Starting route optimization...
Pre-calculating distances...


/var/folders/sd/j5p08m515kd7n9549lkkz_q40000gn/T/ipykernel_38072/1023723280.py:43: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  distance = sum(ox.utils_graph.get_route_edge_attributes(self.dallas_graph_proj, route, 'length'))
/var/folders/sd/j5p08m515kd7n9549lkkz_q40000gn/T/ipykernel_38072/1023723280.py:43: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  distance = sum(ox.utils_graph.get_route_edge_attributes(self.dallas_graph_proj, route, 'length'))
/var/folders/sd/j5p08m515kd7n9549lkkz_q40000gn/T/ipykernel_38072/1023723280.py:43: FutureWarning: The `get_route_edge_attributes` function h


Route Summary:
Total Stops: 28
Total Distance: 139.77 km (86.85 miles)

Optimized Route:
Stop 0: 5351 Fults Blvd., Dallas, TX
Stop 1: 8332 Moorcroft Drive, Dallas, TX
Stop 2: 8814 Redondo Drive, Dallas, TX
Stop 3: 1412 Moran Drive, Dallas, TX
Stop 4: 11635 Sahara Way, Dallas, TX
Stop 5: 11008 Ridgemeadow Drive, Dallas, TX
Stop 6: 12167 Ridgelake Drive, Dallas, TX
Stop 7: 11322 Cactus Lane, Dallas, TX
Stop 8: 8855 Liptonshire, Dallas, TX
Stop 9: 10223 Clary Drive, Dallas, TX
Stop 10: 9731 Champa Drive, Dallas, TX
Stop 11: 10357 Bel Aire, Dallas, TX
Stop 12: 2418 Montalba Avenue, Dallas, TX
Stop 13: 7318 Crownrich Lane, Dallas, TX
Stop 14: 6218 Velasco Avenue, Dallas, TX
Stop 15: 6211 Annapolis Lane, Dallas, TX
Stop 16: 7042 Desco Drive, Dallas, TX
Stop 17: 3724 Purdue Street, Dallas, TX
Stop 18: 7526 Morton Street, Dallas, TX
Stop 19: 4078 Creekdale Drive, Dallas, TX
Stop 20: 1500 Marilla Street, Dallas, TX
Stop 21: 2925 Lenway Street, Dallas, TX
Stop 22: 3203 Bertrand Avenue, Dallas, 

/var/folders/sd/j5p08m515kd7n9549lkkz_q40000gn/T/ipykernel_38072/1023723280.py:43: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  distance = sum(ox.utils_graph.get_route_edge_attributes(self.dallas_graph_proj, route, 'length'))
/var/folders/sd/j5p08m515kd7n9549lkkz_q40000gn/T/ipykernel_38072/1023723280.py:43: FutureWarning: The `get_route_edge_attributes` function has been deprecated and will be removed in the v2.0.0 release. Use the `routing.route_to_gdf` function instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  distance = sum(ox.utils_graph.get_route_edge_attributes(self.dallas_graph_proj, route, 'length'))
